In [6]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# just for the sake of version support with 2.7

In [7]:
import tensorflow as tf
import numpy as np
import os

In [15]:
# here we define the fn which will take the feature column as an input to the cnn from input fn
# and build our cnn with layers we want

def createCnnModel(features, labels, mode):
    # since we r working with (28,28) mnist image
    # our input should be of shape [batchSize, height, width, channel]
    inputLayer = tf.reshape(features['x'], [-1, 28, 28, 1])
    
    # here -1 is used to represent that it will take dynamically as much there will b in features
    
    # now we ll add our first cnn layer and give it the first input
    
    conv1 = tf.layers.conv2d(inputs = inputLayer, filters = 32, kernel_size = [5,5], padding = 'same',
                            activation = tf.nn.relu)
    # here in padding 'same' or 'valid' means that the output dimension from the filter should be kept the
    # same as input dimension so tf will do thT by adding padding around edges [1,1]
    #output shape [-1, 28, 28, 32]
    
    
    # adding pooling layer after it
    pool1 = tf.layers.max_pooling2d(inputs = conv1, pool_size = [2, 2], strides = 2)
    #output shape = [-1, 14, 14, 32]
    
    # adding one more conv layer
    conv2 = tf.layers.conv2d(inputs = pool1, filters = 64, kernel_size = [5, 5], padding = 'same',
                            activation = tf.nn.relu)
    # output shape = [-1, 14, 14, 64]
    
    pool2 = tf.layers.max_pooling2d(inputs = conv2, pool_size = [2, 2], strides = 2)
    # output shape = [-1, 7, 7, 64]
    
    # now we ll add a dense normal layer of neurons, for this we first need to flatten the output 
    # of the convolutional 
    
    flatLayer = tf.reshape(pool2, [-1, 7 * 7 * 64])
    
    dense = tf.layers.dense(inputs = flatLayer, units = 1024, activation = tf.nn.relu)
    # output dimension = [-1, 1024]
    
    # now add dropout layer to avoid overfitting
    
    dropout = tf.layers.dropout(inputs = dense, rate = 0.4, training = (mode == tf.estimator.ModeKeys.TRAIN))
    
    # here in dropout layer training parameter takes in true or false, dropout will only b
    # applied if training mode is true
    #output dimension = [-1, 1024]
    
    # now we ll take out the final output in 10 classes for 0-9
    # we call it logits layer
    
    logits = tf.layers.dense(inputs = dropout, units = 10)
    #output dimension = [-1, 10]
    
    # here we wont directly use softmax fn to take out probability directly
    # rather we would create a proper dictionary to take out which class as well 
    # as softmax probability
    
    #tf.argmax returns the index of the largest value in the axis
    predictions = {'class' : tf.argmax(logits, axis = 1),
                  'probability' : tf.nn.softmax(logits)}
    
    # We compile our predictions in a dict, and return an EstimatorSpec object:
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode = mode, predictions = predictions)
    
    # now before building estimator for train and evaluate, we need to 
    # calculate loss and all
    
    # first we ll convert the int labels of 0-9 to one hot encoding
    # in keras we have a fn for this
    
    onehotLabels = tf.one_hot(tf.cast(labels, tf.int32), depth = 10)
    
    #loss 
    loss = tf.losses.softmax_cross_entropy(
    onehot_labels=onehotLabels, logits=logits)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        
        optimizer = tf.train.GradientDescentOptimizer(0.001)
        
        train = optimizer.minimize(loss=loss,
        global_step=tf.train.get_global_step())
        
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train)
    
    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
          "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


    
    

In [16]:
# now after cnn building fn is done, we ll make fn to load mnist dataset

mnist = tf.contrib.learn.datasets.load_dataset('mnist')
trainData = mnist.train.images
trainLabels = np.asarray(mnist.train.labels, dtype = np.int32)

testData = mnist.test.images
testLabels = mnist.test.labels

# now we ll build our estimator to use the cnn_model fn to 
# build the cnn

mnistClassifier = tf.estimator.Estimator(model_fn = createCnnModel, model_dir = os.getcwd() + '/model_save/tf_mnist_cnn_model')

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_steps': None, '_tf_random_seed': 1, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_secs': 600, '_save_summary_steps': 100, '_model_dir': '/home/saransh/Desktop/tensorflow/model_save/tf_mnist_cnn_model', '_session_config': None, '_keep_checkpoint_max': 5}


In [17]:
# now we hv loaded the data, we ll go for training

trainInputFn = tf.estimator.inputs.numpy_input_fn(
                x={"x": trainData},
                y=trainLabels,
                batch_size=100,
                num_epochs=None,
                shuffle=True)

mnistClassifier.train(input_fn = trainInputFn, steps = 20000)

# now evaluating model and printing

evalInputFn = tf.estimator.inputs.numpy_input_fn(
    x={"x": testData},
    y= testLabels,
    num_epochs=1,
    shuffle=False)

eval_results = mnistClassifier.evaluate(input_fn=evalInputFn)
print(eval_results)


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /home/saransh/Desktop/tensorflow/model_save/tf_mnist_cnn_model/model.ckpt.
INFO:tensorflow:step = 1, loss = 2.3175
INFO:tensorflow:global_step/sec: 2.35863
INFO:tensorflow:step = 101, loss = 2.29916 (42.399 sec)
INFO:tensorflow:global_step/sec: 2.3991
INFO:tensorflow:step = 201, loss = 2.27669 (41.683 sec)
INFO:tensorflow:global_step/sec: 2.6964
INFO:tensorflow:step = 301, loss = 2.24054 (37.086 sec)
INFO:tensorflow:global_step/sec: 2.79296
INFO:tensorflow:step = 401, loss = 2.22648 (35.804 sec)
INFO:tensorflow:global_step/sec: 2.87325
INFO:tensorflow:step = 501, loss = 2.19839 (34.804 sec)
INFO:tensorflow:global_step/sec: 2.88027
INFO:tensorflow:step = 601, loss = 2.13519 (34.719 sec)
INFO:tensorflow:global_step/sec: 2.81968
INFO:tensorflow:step = 701, loss = 2.11833 (35.465 sec)
INFO:tensorflow:global_step/sec: 2.59792
INFO:tensorflow:step = 801, loss = 2.07455 (38.493 sec)
INFO:tensorflow:globa

INFO:tensorflow:global_step/sec: 2.57451
INFO:tensorflow:step = 7701, loss = 0.225302 (38.842 sec)
INFO:tensorflow:global_step/sec: 2.61778
INFO:tensorflow:step = 7801, loss = 0.0980592 (38.200 sec)
INFO:tensorflow:global_step/sec: 2.43503
INFO:tensorflow:step = 7901, loss = 0.327674 (41.067 sec)
INFO:tensorflow:global_step/sec: 2.62042
INFO:tensorflow:step = 8001, loss = 0.166533 (38.163 sec)
INFO:tensorflow:global_step/sec: 2.61971
INFO:tensorflow:step = 8101, loss = 0.404898 (38.171 sec)
INFO:tensorflow:global_step/sec: 2.52444
INFO:tensorflow:step = 8201, loss = 0.358839 (39.613 sec)
INFO:tensorflow:global_step/sec: 2.52253
INFO:tensorflow:step = 8301, loss = 0.146071 (39.643 sec)
INFO:tensorflow:global_step/sec: 2.68995
INFO:tensorflow:step = 8401, loss = 0.294906 (37.176 sec)
INFO:tensorflow:global_step/sec: 2.66818
INFO:tensorflow:step = 8501, loss = 0.293637 (37.479 sec)
INFO:tensorflow:global_step/sec: 2.66656
INFO:tensorflow:step = 8601, loss = 0.375839 (37.501 sec)
INFO:tens

INFO:tensorflow:step = 15401, loss = 0.0921111 (36.287 sec)
INFO:tensorflow:global_step/sec: 2.65357
INFO:tensorflow:step = 15501, loss = 0.155279 (37.685 sec)
INFO:tensorflow:global_step/sec: 2.63741
INFO:tensorflow:step = 15601, loss = 0.140543 (37.917 sec)
INFO:tensorflow:Saving checkpoints for 15663 into /home/saransh/Desktop/tensorflow/model_save/tf_mnist_cnn_model/model.ckpt.
INFO:tensorflow:global_step/sec: 2.65204
INFO:tensorflow:step = 15701, loss = 0.193225 (37.706 sec)
INFO:tensorflow:global_step/sec: 2.74897
INFO:tensorflow:step = 15801, loss = 0.200703 (36.377 sec)
INFO:tensorflow:global_step/sec: 2.64758
INFO:tensorflow:step = 15901, loss = 0.143752 (37.770 sec)
INFO:tensorflow:global_step/sec: 2.54752
INFO:tensorflow:step = 16001, loss = 0.19291 (39.254 sec)
INFO:tensorflow:global_step/sec: 2.54619
INFO:tensorflow:step = 16101, loss = 0.177647 (39.275 sec)
INFO:tensorflow:global_step/sec: 2.64422
INFO:tensorflow:step = 16201, loss = 0.150235 (37.818 sec)
INFO:tensorflow:

KeyError: 'classes'